# Vineyard spray volume rate I

*[Hugo Pires](https://www.linkedin.com/in/hjmpires), 2016*

## Introduction


In the first notebook of this series, I've started to calculate what is the correct spray volume for a given vineyard. In this one I will make some calculation on vineyards. As you will see, I can reuse some code, but I will need (as expected) some modifications.

## As usual,
I will need some libraries...

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### ...and some data

on [Albuz ATR 80 hollow cone nozzles](http://www.albuz-spray.com/en/busesenceramique-ceramicnozzles-boquillasdeceramica).

In [2]:
COLOR_CODES = ['WHITE','LILAC','BROWN','YELLOW','ORANGE','RED','GREY','GREEN','BLACK','BLUE']

PRESSURES = np.array([5.,6.,7.,8.,9.,10.,11.,12.,13.,14.,15.,16.,17.,18.,19.,20.,21.,22.,23.,24.,25.])

FLOW_RATES = np.array([
       [ 0.27,  0.36,  0.48,  0.73,  0.99,  1.38,  1.5 ,  1.78,  2.  ,  2.45],
       [ 0.29,  0.39,  0.52,  0.8 ,  1.08,  1.51,  1.63,  1.94,  2.18,  2.67],
       [ 0.32,  0.42,  0.56,  0.86,  1.17,  1.62,  1.76,  2.09,  2.35,  2.87],
       [ 0.34,  0.45,  0.6 ,  0.92,  1.24,  1.73,  1.87,  2.22,  2.5 ,  3.06],
       [ 0.36,  0.48,  0.64,  0.97,  1.32,  1.83,  1.98,  2.35,  2.64,  3.24],
       [ 0.38,  0.5 ,  0.67,  1.03,  1.39,  1.92,  2.08,  2.47,  2.78,  3.4 ],
       [ 0.39,  0.52,  0.7 ,  1.07,  1.45,  2.01,  2.17,  2.58,  2.9 ,  3.56],
       [ 0.41,  0.55,  0.73,  1.12,  1.51,  2.09,  2.26,  2.69,  3.03,  3.71],
       [ 0.43,  0.57,  0.76,  1.17,  1.57,  2.17,  2.35,  2.79,  3.14,  3.85],
       [ 0.44,  0.59,  0.79,  1.21,  1.63,  2.25,  2.43,  2.89,  3.26,  3.99],
       [ 0.46,  0.61,  0.81,  1.25,  1.69,  2.33,  2.51,  2.99,  3.36,  4.12],
       [ 0.47,  0.63,  0.84,  1.29,  1.74,  2.4 ,  2.59,  3.08,  3.47,  4.25],
       [ 0.48,  0.64,  0.86,  1.33,  1.79,  2.47,  2.67,  3.17,  3.57,  4.37],
       [ 0.5 ,  0.66,  0.89,  1.37,  1.84,  2.54,  2.74,  3.25,  3.67,  4.49],
       [ 0.51,  0.68,  0.91,  1.4 ,  1.89,  2.6 ,  2.81,  3.34,  3.76,  4.61],
       [ 0.52,  0.7 ,  0.93,  1.44,  1.94,  2.67,  2.88,  3.42,  3.85,  4.72],
       [ 0.54,  0.71,  0.95,  1.48,  1.99,  2.73,  2.95,  3.5 ,  3.94,  4.84],
       [ 0.55,  0.73,  0.98,  1.51,  2.03,  2.79,  3.01,  3.57,  4.03,  4.94],
       [ 0.56,  0.74,  1.  ,  1.54,  2.07,  2.85,  3.07,  3.65,  4.12,  5.05],
       [ 0.57,  0.76,  1.02,  1.58,  2.12,  2.91,  3.14,  3.72,  4.2 ,  5.15],
       [ 0.58,  0.77,  1.04,  1.61,  2.16,  2.97,  3.2 ,  3.8 ,  4.28,  5.25]
    ])

Let's reuse some classes that I've created on the first notebook, with diferent names if needed.

In [3]:
class NozzleType():
    
    def __init__(self, manufacturer, name, model, color_codes, pressures, flow_rates):
        self.manufacturer = manufacturer
        self.name = name
        self.model = model
        self.color_codes = color_codes
        self.pressures = pressures
        self.flow_rates = flow_rates
        
class Vineyard():
    
    def __init__(self, grapevine_height, grapevine_width, row_distance, grapevine_distance, vineyard_area):
        """
        Sets vineyard parameters and calculates the number of plants
        Calculates Wine Row Volume and the amount of spray needed per hectare and all vineyard
        """
        self.grapevine_height, self.grapevine_width = grapevine_height, grapevine_width
        self.row_distance, self.grapevine_distance = row_distance, grapevine_distance
        self.vineyard_area = vineyard_area
        self.grapevine_number = round(10000 / (row_distance * grapevine_distance) * self.vineyard_area)
        self.wine_row_volume = self.grapevine_height * self.grapevine_width * 10000 / self.row_distance
        #And the LWA, too
        self.leaf_wall_area = 2 * self.grapevine_height * 10000 / self.row_distance
        
class Pesticide():
    
    def __init__(self, name, dosis):
        self.name = name
        self.dosis = dosis
        
class Sprayer():
    
    def __init__(self, nozzle_number, tank_capacity):
        self.nozzle_number = nozzle_number
        self.nozzle_array = []
        self.tank_capacity = tank_capacity
        
class Nozzle():
    
    def __init__(self, flow_rate, nozzle_type, sprayer):
        self.flow_rate = flow_rate
        self.nozzle_type = nozzle_type
        self.sprayer = sprayer
        
        #Selecting the right nozzle
        dif = np.abs(self.nozzle_type.flow_rates - self.flow_rate / self.sprayer.nozzle_number)
        pressure_index, nozzle_index = np.unravel_index(dif.argmin(),\
                                                        (self.nozzle_type.flow_rates.shape[0],self.nozzle_type.flow_rates.shape[1]))
        self.color_code = self.nozzle_type.color_codes[nozzle_index]
        self.pressure = self.nozzle_type.pressures[pressure_index]
        self.real_flow_rate = round(self.nozzle_type.flow_rates[pressure_index, nozzle_index],2)
        
class Tractor():
    
    def __init__(self, speed):
        """
        When you have a speedometer, just use it
        """
        self.speed = speed
        
    def set_speed(self, distance, time):
        """
        If not, just make a simple speed test
        Probably we have to make some changes on Imperial Units
        """
        self.speed = 3.6 * distance / time
        
class Application():
    
    def __init__(self, vineyard, pesticide, sprayer, tractor):
        self.vineyard = vineyard
        self.pesticide = pesticide
        self.sprayer = sprayer
        self.tractor = tractor
        
        #The spray volume rate is calculated in a diferent way
        self.spray_volume_rate = 0.095 * self.vineyard.wine_row_volume
        self.spray_volume_total = self.spray_volume_rate * self.vineyard.vineyard_area
        
        #Probably the most important formula
        self.flow_rate = (self.spray_volume_rate * self.tractor.speed * self.vineyard.row_distance) / 600
        self.selected_nozzle = Nozzle(self.flow_rate, albuz_atr80, self.sprayer)
        
        self.concentration = self.pesticide.dosis / self.spray_volume_rate
        self.total_pesticide = self.concentration * self.sprayer.tank_capacity
    
    def get_nozzle_config(self):
        return self.selected_nozzle
    
    def get_tank_mix(self):
        return ('Mix {} L of water with {} g of pesticide in tank'.\
                format(self.sprayer.tank_capacity, round(self.total_pesticide)))
    
    def get_tank_mix_lwa(self):
        self.lwa_dosis = self.pesticide.lwa_dosis * self.vineyard.leaf_wall_area
        return ('Mix {} L of water with {} g of pesticide in tank'.\
                format(self.sprayer.tank_capacity, round(self.total_pesticide)))
        

## Let's spray

A slightly diferent place...

In [4]:
albuz_atr80 = NozzleType('Albuz', 'ATR 80', 'hollow cone', COLOR_CODES, PRESSURES, FLOW_RATES )
old_valley = Vineyard(1.8,0.5,2.8,0.5,1)
special_fungicide = Pesticide('Copper MIX', 2000)
fan_sprayer = Sprayer(10, 1000)
rusty_tractor = Tractor(6)
tomorrow_job = Application(old_valley, special_fungicide, fan_sprayer, rusty_tractor)

Let´s get the flow rate for this application, and the proper nozzle

In [5]:
the_right_nozzle = tomorrow_job.get_nozzle_config()
print(the_right_nozzle.color_code)
print(the_right_nozzle.pressure)
print(the_right_nozzle.flow_rate)
print(tomorrow_job.get_tank_mix())

YELLOW
7.0
8.55
Mix 1000 L of water with 6550 g of pesticide in tank


But in some pesticides, a dosis per LWA unit is available, expressed in ammount of product per 10 000 m<sup>2</sup>

In [6]:
class FuturePesticide():
    
    def __init__(self, name, lwa_dosis):
        self.name = name
        self.lwa_dosis = lwa_dosis
        
class FutureApplication():
    
    def __init__(self, vineyard, future_pesticide, sprayer, tractor):
        self.vineyard = vineyard
        self.future_pesticide = future_pesticide
        self.sprayer = sprayer
        self.tractor = tractor
        
        #The spray volume rate is calculated in a diferent way
        self.spray_volume_rate = 0.095 * self.vineyard.wine_row_volume
        self.spray_volume_total = self.spray_volume_rate * self.vineyard.vineyard_area
        
        #Probably the most important formula
        self.flow_rate = (self.spray_volume_rate * self.tractor.speed * self.vineyard.row_distance) / 600
        self.selected_nozzle = Nozzle(self.flow_rate, albuz_atr80, self.sprayer)
        
        self.lwa_dosis = self.future_pesticide.lwa_dosis * self.vineyard.leaf_wall_area / 10000
        
        self.concentration = self.lwa_dosis / self.spray_volume_rate
        self.total_pesticide = self.concentration * self.sprayer.tank_capacity
    
    def get_nozzle_config(self):
        return self.selected_nozzle
    
    def get_tank_mix_lwa(self):
        
        return ('Mix {} L of water with {} g of pesticide in tank'.\
                format(self.sprayer.tank_capacity, round(self.total_pesticide)))

Let's use a new fungicide with the a lower dosis per LWA

In [7]:
new_fungicide = FuturePesticide('Copper MIX EXTRA', 1000)
fan_sprayer = Sprayer(10, 1000)
rusty_tractor = Tractor(6)
weekend_job = FutureApplication(old_valley, new_fungicide, fan_sprayer, rusty_tractor)

Now, with some changes

In [8]:
the_right_nozzle = weekend_job.get_nozzle_config()
print(the_right_nozzle.color_code)
print(the_right_nozzle.pressure)
print(the_right_nozzle.flow_rate)
print(weekend_job.get_tank_mix_lwa())

YELLOW
7.0
8.55
Mix 1000 L of water with 4211 g of pesticide in tank


## Resources

This work is made of four ingredients: some ideas, lots of discussion with coleagues and farmers, support from [Impactwave](http://www.impactwave.com) team (specially Gonçalo Martins) and all the effort of the Python community:

* [Python 3.5.2 documentation](https://docs.python.org/3/)
* [matplotlib](http://matplotlib.org/)
* [NumPy](http://www.numpy.org/)